Usage of the package
========

First, some imports

In [1]:
import gzip
import pickle
import random
from collections import namedtuple

from territories import Territory, MissingTreeCache

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size. That is why, by default, once created, the tree is stored on disk.

Here we will create a very simple tree out of the **tree.txt** file.

In [2]:
Node = namedtuple('Node', ('id', 'parent_id', 'label', 'level'))
split = lambda x: (arg if arg != 'null' else None for arg in x[:-1].split('; '))

try:
    Territory.load_tree()
except MissingTreeCache:
    with open("tree.txt", "r") as file:
        lines = file.readlines()
        stream = ([Node(*split(x) )for x in lines])
        Territory.build_tree(data_stream=stream, save_tree=False)

Then, you can start to create territories from arbitrary territoiral units.

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [3]:
# some node of the tree
print('\n'.join([f"{e.name} | {e.partition_type}" for e in random.sample(Territory.tree.nodes(), 12)]))

Montamisé | COM
Bénesse-lès-Dax | COM
Morogues | COM
Saint-Jean-de-Maurienne | COM
Margon | COM
Rancourt | COM
Soucia | COM
Crèvecœur-le-Grand | COM
Fos | COM
Champfrémont | COM
Champcerie | COM
Saint-Simon | COM


In [4]:
with open("tree_large.gzip", "rb") as file:
    lines = pickle.loads(gzip.decompress(file.read()))

stream = ([Node(*split(x) )for x in lines])
Territory.build_tree(data_stream=stream, save_tree=False)

a = Territory.from_name("COM:69123", "COM:93055", "COM:94052")
b = Territory.from_name("COM:27429", "REG:84", "DEP:75")
c = Territory.from_name("COM:38185", "COM:31555", "REG:11")
d = Territory.from_name("COM:33063", "COM:13055", "REG:28")
e = Territory.from_name("COM:35238", "COM:35047", "DEP:27")
f = Territory.from_name("COM:59350", "COM:38442", "REG:53")

## Operations on entities

Usual operation on territories works as expected :

In [5]:
# addition

print(a, c)
print(a + c) # This simplify to France

Pantin|Lyon|Nogent-sur-Marne Toulouse|Grenoble|Île-de-France
Toulouse|Grenoble|Lyon|Île-de-France


In [6]:
# substraction

print(a, d)
print(a - d) # only Saint Etienne remains

Pantin|Lyon|Nogent-sur-Marne Normandie|Bordeaux|Marseille
Pantin|Lyon|Nogent-sur-Marne


More importantly, sets operations are also supported

In [7]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Pantin|Lyon|Nogent-sur-Marne and Normandie|Bordeaux|Marseille is {}
Union of Toulouse|Grenoble|Île-de-France and Bretagne|Lille|Saint-Pierre-de-Chartreuse is Île-de-France|Saint-Pierre-de-Chartreuse|Toulouse|Bretagne|Grenoble|Lille


Territorial units may have parents or children, but Territory do not. As a territory may be formed of several territorial units, it has a LCA, a Lowest Common Ancestor.

In [15]:
f.lowest_common_ancestor()

France

You can easily retrieve all ancestors of a territory with the `.ancestors()` method :

In [16]:
a.ancestors()

{Auvergne-Rhône-Alpes,
 France,
 Rhône,
 Seine-Saint-Denis,
 Val-de-Marne,
 Île-de-France}